In [43]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error # poner este como metrica
import matplotlib.pyplot as plt
import seaborn as sns 
from os import listdir
from os.path import isfile, join
from datetime import date, time, datetime
from pyparsing import Path
from fuzzywuzzy import process
from fuzzywuzzy import fuzz 



In [44]:
ds_path = Path(".\Datasets")
Lista_Localidad= pd.DataFrame()
Lista_Municipios= pd.DataFrame()
Lista_Departamentos= pd.DataFrame()
Lista_provincias= pd.DataFrame()
df = pd.read_csv('.\Argentina\provincias.csv',delimiter = ',',decimal =",", encoding="UTF-8") 
df1= pd.read_csv('.\Argentina\departamentos.csv',delimiter = ',',decimal =",", encoding="UTF-8") 
df2= pd.read_csv('.\Argentina\municipios.csv',delimiter = ',',decimal =",", encoding="UTF-8") 
df3= pd.read_csv('.\Argentina\localidades-censales.csv',delimiter = ',',decimal =",", encoding="UTF-8") 
Lista_Localidad['Nombre_localidad'] = df3['nombre'].dropna().values
Lista_Municipios['Nombre_Municipio'] = df2['nombre'].dropna().values
Lista_Departamentos['Nombre_Departamentos'] = df1['nombre'].dropna().values
Lista_provincias['Nombre_provincias'] = df['nombre'].dropna().values

nombres=[]
proporcion=[]
def cambios ( mal, bien):
    for i in mal:
        x=process.extractOne(i,bien)
        nombres.append(x[0])
        proporcion.append(x[1])
    return(nombres, proporcion)

In [ ]:
map_list = []

def rename ( mal, bien):
    for name in mal:
        best_ratio = None
        for idx, surname in enumerate(bien):
            if best_ratio == None:
                best_ratio = fuzz.ratio(name, surname)
                best_idx = 0
            else:
                ratio = fuzz.ratio(name, surname)
                if  ratio > best_ratio:
                    best_ratio = ratio
                    best_idx = idx
        map_list.append(bien[best_idx]) # obtain surname

#dico = dico[["Name", "Surname", "Age", "Studies"]] # reorder columns

In [49]:
ds_path = Path(".\Datasets")
Arc_Localidades=[]
for file in ds_path.glob('localidades*'):
    df = pd.read_csv(file,decimal =",") 
    Arc_Localidades.append(df)
Localidades=pd.concat(Arc_Localidades)

Localidades = Localidades.rename(columns={'centroide_lon':'Longitud','centroide_lat':'Latitud', 'id':'Id_Localidades', 'categoria':'Categoria', 'departamento_id':'Departamento_Id', 'departamento_nombre':'Departamento_Nombre',
'fuente':'Fuente','localidad_censal_id':'Localidad_Censal_Id','localidad_censal_nombre':'Localidad_Censal_Nombre','municipio_id':'Municipio_Id','municipio_nombre':'Municipio_Nombre','nombre':'Nombre','provincia_id':'Provincia_Id','provincia_nombre':'Provincia_Nombre' })
Localidades['Departamento_Id'] = Localidades['Departamento_Id'].replace(np.NaN,"1")
Localidades['Municipio_Id'] = Localidades['Municipio_Id'].replace(np.NaN,"1")
Localidades = Localidades.astype({'Departamento_Id':'double','Municipio_Id':'double'})
Localidades = Localidades.astype({'Departamento_Id':'int64','Municipio_Id':'int64'})
Localidades['Longitud']= pd.to_numeric(Localidades['Longitud'],errors='coerce')
Localidades['Longitud'] = Localidades['Longitud'].astype(float)
Localidades['Latitud']= pd.to_numeric(Localidades['Latitud'],errors='coerce')
Localidades['Latitud'] = Localidades['Latitud'].astype(float)
Localidades_final = Localidades.copy()

Provincia = Localidades.loc[:,['Provincia_Id','Provincia_Nombre']]
Provincia = Provincia.drop_duplicates()
#Provincia = Provincia.set_index('Provincia_Id')


Departamento = Localidades.loc[:,['Departamento_Id','Departamento_Nombre','Provincia_Id']]
Departamento = Departamento.drop_duplicates()
#Departamento = Departamento.set_index('Departamento_Id')


Municipio = Localidades.loc[:,['Municipio_Id','Municipio_Nombre','Departamento_Id']]
Municipio = Municipio.drop_duplicates()
#Municipio = Municipio.set_index('Municipio_Id')

Localidad = Localidades.loc[:,['Localidad_Censal_Id','Localidad_Censal_Nombre','Municipio_Id']]
Localidad = Localidad.drop_duplicates()
#Localidad = Localidad.set_index('Localidad_Censal_Id')

Localidades_final.drop(['Provincia_Nombre','Departamento_Nombre','Municipio_Nombre','Localidad_Censal_Nombre'], axis = 1, inplace = True)

#Localidades_final = Localidades_final.set_index('Id_Localidades')


In [73]:
Arc_Clientes=[]
for file in ds_path.glob('Cliente*'):
    df = pd.read_csv(file,delimiter = ';',decimal =",", encoding="UTF-8") 
    Arc_Clientes.append(df)
Clientes=pd.concat(Arc_Clientes)
Clientes.drop('col10', axis = 1, inplace = True)
Clientes['Y'] = Clientes['Y'].str.replace(',','.')
Clientes['X'] = Clientes['X'].str.replace(',','.')
Clientes  = Clientes.rename(columns={'X':'Longitud','Y':'Latitud','ID':'Id_Cliente'})
Clientes['Longitud']= pd.to_numeric(Clientes['Longitud'],errors='coerce')
Clientes['Longitud'] = Clientes['Longitud'].astype(float)
Clientes['Latitud']= pd.to_numeric(Clientes['Latitud'],errors='coerce')
Clientes['Latitud'] = Clientes['Latitud'].astype(float)
Clientes['Localidad'] = Clientes['Localidad'].str.capitalize() 
#Clientes = Clientes.set_index('Id_Cliente')
#Clientes.to_sql(name='Clientes', con=conexion, if_exists='append')
Clientes.dtypes

Id_Cliente             int64
Provincia             object
Nombre_y_Apellido     object
Domicilio             object
Telefono              object
Edad                 float64
Localidad             object
Longitud             float64
Latitud              float64
dtype: object

In [52]:
Provincia.dtypes
#Departamento.dtypes
#Municipio.dtypes
#Localidad.dtypes
#Localidades_final.dtypes


Provincia_Id         int64
Provincia_Nombre    object
dtype: object

In [61]:
Lista_provincias.sort_values('Nombre_provincias').head(2)

,Nombre_provincias
22,Buenos Aires
10,Catamarca


In [84]:
# correccion provincias con funcion cambios

mal = Clientes['Provincia'].dropna().values
bien = Provincia['Provincia_Nombre'].dropna().values
#Lista_provincias['Nombre_provincias']
#Provincia['Provincia_Nombre']
corregido , rango=cambios(mal, bien)

# se crea una nueva columna para el data frame
Clientes['Provincia_Nombre']=pd.Series(corregido)



In [ ]:
# correccion provincias con funcion rename
mal = Clientes['Provincia'].dropna().values
bien = Provincia['Provincia_Nombre'].dropna().values
#Lista_provincias['Nombre_provincias']
#Provincia['Provincia_Nombre']
corregido , rango=rename(mal, bien)
# se crea una nueva columna para el data frame
Clientes['Provincia_Nombre'] = pd.Series(map_list) # add column

In [85]:
Arc_Clientes=[]
for file in ds_path.glob('Cliente*'):
    df = pd.read_csv(file,delimiter = ';',decimal =",", encoding="UTF-8") 
    Arc_Clientes.append(df)
Clientes=pd.concat(Arc_Clientes)
Clientes.drop('col10', axis = 1, inplace = True)
Clientes['Y'] = Clientes['Y'].str.replace(',','.')
Clientes['X'] = Clientes['X'].str.replace(',','.')
Clientes  = Clientes.rename(columns={'X':'Longitud','Y':'Latitud','ID':'Id_Cliente'})
Clientes['Longitud']= pd.to_numeric(Clientes['Longitud'],errors='coerce')
Clientes['Longitud'] = Clientes['Longitud'].astype(float)
Clientes['Latitud']= pd.to_numeric(Clientes['Latitud'],errors='coerce')
Clientes['Latitud'] = Clientes['Latitud'].astype(float)
Clientes['Localidad'] = Clientes['Localidad'].str.capitalize() 
Clientes = Clientes.set_index('Id_Cliente')
#Clientes.to_sql(name='Clientes', con=conexion, if_exists='append')
Clientes.dtypes

Provincia             object
Nombre_y_Apellido     object
Domicilio             object
Telefono              object
Edad                 float64
Localidad             object
Longitud             float64
Latitud              float64
dtype: object

In [134]:
#dfResult =  Localidades.merge(Clientes, left_on='Provincia_Nombre', right_on='Provincia', how='inner')
#dfResult
 

In [86]:
Arc_TipoGastos=[]
for file in ds_path.glob('TiposDeGasto*'):
    df = pd.read_csv(file)
    Arc_TipoGastos.append(df)
Tipo_Gastos=pd.concat(Arc_TipoGastos)
Tipo_Gastos = Tipo_Gastos.set_index('IdTipoGasto')
Tipo_Gastos.dtypes

Descripcion         object
Monto_Aproximado     int64
dtype: object

In [124]:
Arc_Sucursales=[]
for file in ds_path.glob('Sucursales*'):
    df = pd.read_csv(file,delimiter = ';')
    Arc_Sucursales.append(df)
Sucursales=pd.concat(Arc_Sucursales)
Sucursales['Latitud'] = Sucursales['Latitud'].str.replace(',','.')
Sucursales['Longitud'] = Sucursales['Longitud'].str.replace(',','.')
Sucursales = Sucursales.astype({'Longitud':'float','Latitud':'float'})
Sucursales  = Sucursales.rename(columns={'ID':'Id_Sucursal'})
#Sucursales['Sum'] = Sucursales['Longitud'] + Sucursales['Latitud']
Sucursales['Localidad']=Sucursales['Localidad'].replace(['Ciudad de Buenos Aires', 'CABA', 'Capital', 'Capital Federal', 'CapFed', 'Cap. Fed.', 'Cap.   Federal', 'Cdad de Buenos Aires'], 'Ciudad Autónoma de Buenos Aires')
Sucursales['Localidad']=Sucursales['Localidad'].replace(['Coroba', 'Cordoba'],'Córdoba')
Sucursales['Provincia']=Sucursales['Provincia'].replace(['Ciudad de Buenos Aires', 'CABA', 'C deBuenos Aires', 'Bs As', 'Bs.As. ', 'B. Aires', 'B.Aires', 'Provincia de Buenos Aires', 'Prov de Bs As.', 'Pcia Bs AS'], 'Buenos Aires')
Sucursales['Provincia']=Sucursales['Provincia'].replace(['Coroba', 'Cordoba'],'Córdoba')
Sucursales = Sucursales.set_index('Id_Sucursal')
Sucursales

,Sucursal,Direccion,Localidad,Provincia,Latitud,Longitud
Id_Sucursal,,,,,,
1,Cabildo,Av. Cabildo 1342,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.567806,-58.449572
2,Palermo 1,Guatemala 5701,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.579035,-58.433566
3,Palermo 2,Gral. Lucio Norberto Mansilla 2668,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.595966,-58.405150
4,Corrientes,Av. Corrientes 2352,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.604685,-58.398764
5,Almagro,Venezuela 3650,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.617308,-58.416179
6,Caballito,Av. Rivadavia 4708,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.616303,-58.431849
7,Flores,Av. Rivadavia 5746,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.622868,-58.446449
8,Alberdi,Av. Juan Bautista Alberdi 1634,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.627306,-58.451498
9,Deposito,Pedernera 530,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.635134,-58.461513


In [128]:
dfResult =  Sucursales.merge(Provincia, left_on='Provincia', right_on='Provincia_Nombre', how='inner')
dfResult

,Sucursal,Direccion,Localidad,Provincia,Latitud,Longitud,Provincia_Id,Provincia_Nombre
0,Cabildo,Av. Cabildo 1342,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.567806,-58.449572,6,Buenos Aires
1,Palermo 1,Guatemala 5701,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.579035,-58.433566,6,Buenos Aires
2,Palermo 2,Gral. Lucio Norberto Mansilla 2668,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.595966,-58.405150,6,Buenos Aires
3,Corrientes,Av. Corrientes 2352,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.604685,-58.398764,6,Buenos Aires
4,Almagro,Venezuela 3650,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.617308,-58.416179,6,Buenos Aires
5,Caballito,Av. Rivadavia 4708,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.616303,-58.431849,6,Buenos Aires
6,Flores,Av. Rivadavia 5746,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.622868,-58.446449,6,Buenos Aires
7,Alberdi,Av. Juan Bautista Alberdi 1634,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.627306,-58.451498,6,Buenos Aires
8,Deposito,Pedernera 530,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.635134,-58.461513,6,Buenos Aires
9,Velez,Av. Juan Bautista Justo 7738,Ciudad Autónoma de Buenos Aires,Buenos Aires,-34.631257,-58.498063,6,Buenos Aires


In [125]:
Provincia

,Provincia_Nombre
Provincia_Id,
6,Buenos Aires
10,Catamarca
14,Córdoba
18,Corrientes
22,Chaco
26,Chubut
30,Entre Ríos
34,Formosa
38,Jujuy


In [88]:
Arc_Gastos=[]
for file in ds_path.glob('Gasto*'):
    df = pd.read_csv(file,delimiter = ',')
    Arc_Gastos.append(df)
Gastos=pd.concat(Arc_Gastos)
Gastos['Fecha'] = pd.to_datetime(Gastos['Fecha'])
Gastos = Gastos.set_index('IdGasto')
Gastos.dtypes

IdSucursal              int64
IdTipoGasto             int64
Fecha          datetime64[ns]
Monto                 float64
dtype: object

In [80]:
Arc_Proveedores=[]
for file in ds_path.glob('Proveedores*'):
    df = pd.read_csv(file,delimiter = ',',decimal =",", encoding="ansi") 
    Arc_Proveedores.append(df)
Proveedores=pd.concat(Arc_Proveedores)
Proveedores['State']=Proveedores['State'].replace('CABA', 'BUENOS AIRES')
Proveedores['City'] = Proveedores['City'].str.capitalize() 
Proveedores['State'] = Proveedores['State'].str.capitalize() 
Proveedores['Country'] = Proveedores['Country'].str.capitalize() 
Proveedores['departamen'] = Proveedores['departamen'].str.capitalize() 
Proveedores = Proveedores.set_index('IDProveedor')
Proveedores.dtypes

Nombre        object
Address       object
City          object
State         object
Country       object
departamen    object
dtype: object

In [84]:
Arc_Compras=[]
for file in ds_path.glob('Compra*'):
    df = pd.read_csv(file,delimiter = ',',decimal =",", encoding="UTF-8") 
    Arc_Compras.append(df)
Compras=pd.concat(Arc_Compras)
Compras['Fecha'] = pd.to_datetime(Compras['Fecha'])
Compras = Compras.set_index('IdCompra')
Compras.dtypes

Fecha            datetime64[ns]
Fecha_Año                 int64
Fecha_Mes                 int64
Fecha_Periodo             int64
IdProducto                int64
Cantidad                  int64
Precio                   object
IdProveedor               int64
dtype: object

In [89]:
Arc_CanalVenta=[]
for file in ds_path.glob('Canal*'):
    df = pd.read_excel(file,decimal =",") 
    Arc_CanalVenta.append(df)
CanalVenta=pd.concat(Arc_CanalVenta)
CanalVenta  = CanalVenta.rename(columns={'CODIGO':'IdCanal'})
CanalVenta = CanalVenta.set_index('IdCanal')
CanalVenta.dtypes

DESCRIPCION    object
dtype: object

In [93]:
Arc_Ventas=[]
for file in ds_path.glob('Venta*'):
    df = pd.read_csv(file,delimiter = ',',decimal =",", encoding="UTF-8") 
    Arc_Ventas.append(df)
Ventas=pd.concat(Arc_Ventas)
Ventas['Fecha'] = pd.to_datetime(Ventas['Fecha'])
Ventas['Fecha_Entrega'] = pd.to_datetime(Ventas['Fecha_Entrega'])
Ventas = Ventas.set_index('IdVenta')
Ventas.dtypes

Fecha            datetime64[ns]
Fecha_Entrega    datetime64[ns]
IdCanal                   int64
IdCliente                 int64
IdSucursal                int64
IdEmpleado                int64
IdProducto                int64
Precio                   object
Cantidad                float64
dtype: object

In [141]:
#dfResult =  Proveedores.merge(Localidades, left_on='City', right_on='Nombre', how='inner')
#dfResult

estilos de join en data frame
dfResult =  Sucursales.merge(Localidades_final, left_on='Sum', right_on='Sum', how='inner')
dfResult

#Localidades_final
concatenar_inner = pd.concat([Sucursales,Localidades_final],join="inner")
concatenar_inner

left_join = pd.merge(Sucursales,Localidades_final, how='inner', left_on='Sum', right_on='Sum')
left_join

In [157]:
Arc_Clientes=[]
type(Arc_Clientes)

list

In [120]:

Localidad = Localidad.drop_duplicates()
Localidad

,Localidad_Censal_Nombre,Municipio_Id
Localidad_Censal_Id,,
6021010,Alberti,60021
6021020,Coronel Seguí,60021
6021030,Mechita,60021
6021040,Pla,60021
6021050,Villa Grisolía,60021
...,...,...
90119030,Yerba Buena - Marcos Paz,908560
94008010,Río Grande,940007
94008020,Tolhuin,942007


In [122]:
Municipio = Municipio.drop_duplicates()
Municipio

,Municipio_Nombre,Departamento_Id
Municipio_Id,,
60021,Alberti,6021
60028,Almirante Brown,6028
60035,Avellaneda,6035
60042,Ayacucho,6042
60049,Azul,6049
...,...,...
1,Capital Federal,2105
940007,Río Grande,94008
942007,Tolhuin,94008


In [106]:
Departamento.dtypes

Departamento_Nombre    object
Provincia_Id            int64
dtype: object

In [114]:
Provincia

,Provincia_Nombre
Provincia_Id,
6,Buenos Aires
10,Catamarca
14,Córdoba
18,Corrientes
22,Chaco
26,Chubut
30,Entre Ríos
34,Formosa
38,Jujuy


In [123]:
dfResult = Municipio.merge(Localidad, left_on='Municipio_Id', right_on='Municipio_Id', how='inner')
dfResult = dfResult.drop_duplicates()
dfResult

,Municipio_Id,Municipio_Nombre,Departamento_Id,Localidad_Censal_Nombre
0,60021,Alberti,6021,Alberti
1,60021,Alberti,6021,Coronel Seguí
2,60021,Alberti,6021,Mechita
3,60021,Alberti,6021,Pla
4,60021,Alberti,6021,Villa Grisolía
...,...,...,...,...
63209,908644,Cevil Redondo,90119,Barrio San José III
63210,908644,Cevil Redondo,90119,Villa Carmela
63211,940007,Río Grande,94008,Río Grande
63212,942007,Tolhuin,94008,Tolhuin


In [115]:
dfResult = Municipio.merge(Localidad, left_on='Municipio_Id', right_on='Municipio_Id', how='inner')
dfResult1 = dfResult.merge(Departamento, left_on='Departamento_Id', right_on='Departamento_Id', how='inner')
dfResult2 = dfResult1.merge(Provincia, left_on='Provincia_Id', right_on='Provincia_Id', how='inner')
dfResult2

,Localidad_Censal_Nombre,Municipio_Id,Municipio_Nombre,Departamento_Id,Departamento_Nombre,Provincia_Id,Provincia_Nombre
0,Alberti,60021,Alberti,6021,Alberti,6,Buenos Aires
1,Coronel Seguí,60021,Alberti,6021,Alberti,6,Buenos Aires
2,Mechita,60021,Alberti,6021,Alberti,6,Buenos Aires
3,Pla,60021,Alberti,6021,Alberti,6,Buenos Aires
4,Villa Grisolía,60021,Alberti,6021,Alberti,6,Buenos Aires
...,...,...,...,...,...,...,...
64761,Villa de Trancas,900126,Municipalidad de Trancas,90112,Trancas,90,Tucumán
64762,Barrio San José III,908560,Municipalidad de Yerba Buena,90119,Yerba Buena,90,Tucumán
64763,Yerba Buena - Marcos Paz,908560,Municipalidad de Yerba Buena,90119,Yerba Buena,90,Tucumán
64764,Barrio San José III,908644,Cevil Redondo,90119,Yerba Buena,90,Tucumán


In [116]:
#Provincia.dtypes
#Departamento.dtypes
#Municipio.dtypes
#Localidad.dtypes
#Localidades_final.dtypes
Clientes
#Tipo_Gastos.dtypes
#Sucursales
#Gastos.dtypes
#Proveedores.dtypes
#Compras.dtypes
#CanalVenta.dtypes
#Ventas.dtypes

,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,Longitud,Latitud
Id_Cliente,,,,,,,,
1,Buenos Aires,HEBER JONI SANTANA,LAS HERAS Y BAT. 24 DE FEBRERO 4150 RINCON DE...,42-5161,58.0,Loma verde,-58.818503,-34.309971
2,Buenos Aires,ANA SAPRIZA,PUEYRREDON Y DUPUY RUTA3 KM 52.500 S/N BÂº LO...,49-7578,61.0,Santa rosa,-58.730738,-34.939083
3,Buenos Aires,FERNANDO LUIS SARALEGUI,CALDERON DE LA BARCA 498,49-3435,15.0,Torres,-59.127941,-34.430822
4,Buenos Aires,MANUELA SARASOLA,"RUTA 36 KM 45, 500 S/N EL PELIGRO",49-2883,29.0,Ruta sol,-58.143940,-34.920527
5,Buenos Aires,MARIO RAÚL SARASUA,492 Y 186 S/N COLONIA URQUIZA,491-4608,34.0,Jose melchor romero,-58.893810,-34.944447
...,...,...,...,...,...,...,...,...
2626,Córdoba,ALICIA MARILEN PETROCELLI,CALLE PUBLICA S/NÂº LA CARBONADA RUTA 9 SUR KM...,156786618,19.0,Cordoba,NaN,NaN
2636,Córdoba,RODOLFO WASHINGTON PICCINI,CALLE PUBLICA SOL NACIENTE BARRIO SOL NACIENTE,NaN,44.0,Cordoba,NaN,NaN
2637,Córdoba,ALFREDO PICERNO,Calle publica LA FLORESTA SUR BARRIO LA FLORE...,4900752,21.0,Cordoba,NaN,NaN


In [118]:
dfResult3 = dfResult2.merge(Clientes, left_on='Provincia_Nombre', right_on='Provincia', how='inner')
dfResult3

,Localidad_Censal_Nombre,Municipio_Id,Municipio_Nombre,Departamento_Id,Departamento_Nombre,Provincia_Id,Provincia_Nombre,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,Longitud,Latitud
0,Alberti,60021,Alberti,6021,Alberti,6,Buenos Aires,Buenos Aires,HEBER JONI SANTANA,LAS HERAS Y BAT. 24 DE FEBRERO 4150 RINCON DE...,42-5161,58.0,Loma verde,-58.818503,-34.309971
1,Alberti,60021,Alberti,6021,Alberti,6,Buenos Aires,Buenos Aires,ANA SAPRIZA,PUEYRREDON Y DUPUY RUTA3 KM 52.500 S/N BÂº LO...,49-7578,61.0,Santa rosa,-58.730738,-34.939083
2,Alberti,60021,Alberti,6021,Alberti,6,Buenos Aires,Buenos Aires,FERNANDO LUIS SARALEGUI,CALDERON DE LA BARCA 498,49-3435,15.0,Torres,-59.127941,-34.430822
3,Alberti,60021,Alberti,6021,Alberti,6,Buenos Aires,Buenos Aires,MANUELA SARASOLA,"RUTA 36 KM 45, 500 S/N EL PELIGRO",49-2883,29.0,Ruta sol,-58.143940,-34.920527
4,Alberti,60021,Alberti,6021,Alberti,6,Buenos Aires,Buenos Aires,MARIO RAÚL SARASUA,492 Y 186 S/N COLONIA URQUIZA,491-4608,34.0,Jose melchor romero,-58.893810,-34.944447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18435755,Villa Ascasubi,141505,Villa Ascasubi,14161,Tercero Arriba,14,Córdoba,Córdoba,ALICIA MARILEN PETROCELLI,CALLE PUBLICA S/NÂº LA CARBONADA RUTA 9 SUR KM...,156786618,19.0,Cordoba,NaN,NaN
18435756,Villa Ascasubi,141505,Villa Ascasubi,14161,Tercero Arriba,14,Córdoba,Córdoba,RODOLFO WASHINGTON PICCINI,CALLE PUBLICA SOL NACIENTE BARRIO SOL NACIENTE,NaN,44.0,Cordoba,NaN,NaN
18435757,Villa Ascasubi,141505,Villa Ascasubi,14161,Tercero Arriba,14,Córdoba,Córdoba,ALFREDO PICERNO,Calle publica LA FLORESTA SUR BARRIO LA FLORE...,4900752,21.0,Cordoba,NaN,NaN
18435758,Villa Ascasubi,141505,Villa Ascasubi,14161,Tercero Arriba,14,Córdoba,Córdoba,DIEGO ENRIQUE PIÑEIRO,MONSEÃ?OR D'ANDREA 1044 ALTO ALBERDI BÂ° ALTO ...,4347875,15.0,Cordoba,NaN,NaN


In [ ]:
#data frame with
#from fuzzywuzzy import process
ok={"Mexico","Argentina", "Chile"}
usuario ="mex f"
aprox =process.extract0ne(usuario,ok)
print("la palabra mas cerca a :",usuario)
print(aprox)
print("es", aprox[0])
print("Con", aprox[1])
df
nombres=[]
proporcion=[]
def cambios ( mal, bien):
    for i in mal:
        x=process.extract0ne(i,bien)
        nombres.append(x[0])
        proporcion.append(x[0])
    return(nombres, proporcion)
mal = df['mal'].dropna().values
bien = df['bien'].dropna().values

corregido, rango=cambios(mal, bien)
# se crea una nueva columna para el data frame
df['correccion']=pd.Series(corregido)
df['ration']=pd.Series(rango)

In [35]:


Lista_Localidad= pd.DataFrame()
Lista_Municipios= pd.DataFrame()
Lista_Departamentos= pd.DataFrame()
Lista_provincias= pd.DataFrame()
df = pd.read_csv('.\Argentina\provincias.csv',delimiter = ',',decimal =",", encoding="UTF-8") 
df1= pd.read_csv('.\Argentina\departamentos.csv',delimiter = ',',decimal =",", encoding="UTF-8") 
df2= pd.read_csv('.\Argentina\municipios.csv',delimiter = ',',decimal =",", encoding="UTF-8") 
df3= pd.read_csv('.\Argentina\localidades-censales.csv',delimiter = ',',decimal =",", encoding="UTF-8") 
Lista_Localidad['Nombre_localidad'] = df3['nombre'].dropna().values
Lista_Municipios['Nombre_Municipio'] = df2['nombre'].dropna().values
Lista_Departamentos['Nombre_Departamentos'] = df1['nombre'].dropna().values
Lista_provincias['Nombre_provincias'] = df['nombre'].dropna().values

nombres=[]
proporcion=[]
def cambios ( mal, bien):
    for i in mal:
        x=process.extractOne(i,bien)
        nombres.append(x[0])
        proporcion.append(x[1])
    return(nombres, proporcion)


,Nombre_provincias
0,Misiones
1,San Luis
2,San Juan
3,Entre Ríos
4,Santa Cruz
5,Río Negro
6,Chubut
7,Córdoba
8,Mendoza
9,La Rioja


In [23]:
Lista_Localidad

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [34]:
df

,categoria,centroide_lat,centroide_lon,fuente,id,iso_id,iso_nombre,nombre,nombre_completo
0,Provincia,-26.8753965086829,-54.6516966230371,IGN,54,AR-N,Misiones,Misiones,Provincia de Misiones
1,Provincia,-33.7577257449137,-66.0281298195836,IGN,74,AR-D,San Luis,San Luis,Provincia de San Luis
2,Provincia,-30.8653679979618,-68.8894908486844,IGN,70,AR-J,San Juan,San Juan,Provincia de San Juan
3,Provincia,-32.0588735436448,-59.2014475514635,IGN,30,AR-E,Entre Ríos,Entre Ríos,Provincia de Entre Ríos
4,Provincia,-48.8154851827063,-69.9557621671973,IGN,78,AR-Z,Santa Cruz,Santa Cruz,Provincia de Santa Cruz
5,Provincia,-40.4057957178801,-67.229329893694,IGN,62,AR-R,Río Negro,Río Negro,Provincia de Río Negro
6,Provincia,-43.7886233529878,-68.5267593943345,IGN,26,AR-U,Chubut,Chubut,Provincia del Chubut
7,Provincia,-32.142932663607,-63.8017532741662,IGN,14,AR-X,Córdoba,Córdoba,Provincia de Córdoba
8,Provincia,-34.6298873058957,-68.5831228183798,IGN,50,AR-M,Mendoza,Mendoza,Provincia de Mendoza
9,Provincia,-29.685776298315,-67.1817359694432,IGN,46,AR-F,La Rioja,La Rioja,Provincia de La Rioja


In [10]:
Localidades_copia

,categoria,centroide_lat,centroide_lon,departamento_id,departamento_nombre,fuente,id,localidad_censal_id,localidad_censal_nombre,municipio_id,municipio_nombre,nombre,provincia_id,provincia_nombre
0,Localidad simple,-35.03307343,-60.28061973,6021.0,Alberti,INDEC,6021010000,6021010,Alberti,60021.0,Alberti,ALBERTI,6,Buenos Aires
1,Localidad simple,-34.868119,-60.39397088,6021.0,Alberti,INDEC,6021020000,6021020,Coronel Seguí,60021.0,Alberti,CORONEL SEGUI,6,Buenos Aires
2,Componente de localidad compuesta,-35.06801367,-60.40259716,6021.0,Alberti,INDEC,6021030000,6021030,Mechita,60021.0,Alberti,MECHITA,6,Buenos Aires
3,Localidad simple,-35.12438198,-60.22006126,6021.0,Alberti,INDEC,6021040000,6021040,Pla,60021.0,Alberti,PLA,6,Buenos Aires
4,Localidad simple,-35.10961783,-60.07055132,6021.0,Alberti,INDEC,6021050000,6021050,Villa Grisolía,60021.0,Alberti,VILLA GRISOLIA,6,Buenos Aires
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4137,Localidad simple,-34.68821595,-58.95742971,6364.0,General Rodríguez,INDEC,6364030005,6364030,General Rodríguez,60364.0,General Rodríguez,BARRIO RUTA 24 KILOMETRO 10,6,Buenos Aires
4138,Localidad simple,-53.78704734,-67.71323507,94008.0,Río Grande,INDEC,94008010000,94008010,Río Grande,940007.0,Río Grande,RIO GRANDE,94,"Tierra del Fuego, Antártida e Islas del Atlánt..."
4139,Localidad simple,-54.51138675,-67.19580446,94008.0,Río Grande,INDEC,94008020000,94008020,Tolhuin,942007.0,Tolhuin,TOLHUIN,94,"Tierra del Fuego, Antártida e Islas del Atlánt..."
4140,Localidad simple,-54.63768608,-67.76694086,94015.0,Ushuaia,INDEC,94015010000,94015010,Laguna Escondida,NaN,NaN,LAGUNA ESCONDIDA,94,"Tierra del Fuego, Antártida e Islas del Atlánt..."


In [14]:
nombres=[]
proporcion=[]
def cambios ( mal, bien):
    for i in mal:
        x=process.extractOne(i,bien)
        nombres.append(x[0])
        proporcion.append(x[1])
    return(nombres, proporcion)

In [68]:
dico = {'Name': ['Arthur','Henri','Lisiane','Patrice'],
        "Age": ["20","18","62","73"],
        "Studies": ['Economics','Maths','Psychology','Medical']
             }
dico2 = {'Surname': ['Henri2','Arthur1','Patrice4','Lisiane3']}

dico = pd.DataFrame.from_dict(dico)
dico2 = pd.DataFrame.from_dict(dico2)


In [77]:
dico.dtypes

Name       object
Age        object
Studies    object
Surname    object
dtype: object

In [78]:
dico2.dtypes

Surname    object
dtype: object

In [76]:
map_list = []
for name in dico['Name']:
    best_ratio = None
    for idx, surname in enumerate(dico2['Surname']):
        if best_ratio == None:
            best_ratio = fuzz.ratio(name, surname)
            best_idx = 0
        else:
            ratio = fuzz.ratio(name, surname)
            if  ratio > best_ratio:
                best_ratio = ratio
                best_idx = idx
    map_list.append(dico2['Surname'][best_idx]) # obtain surname

dico['Surname'] = pd.Series(map_list) # add column
#dico = dico[["Name", "Surname", "Age", "Studies"]] # reorder columns

In [82]:
map_list = []
mal = Clientes['Provincia'].dropna().values
bien = Provincia['Provincia_Nombre'].dropna().values

for name in mal:
    best_ratio = None
    for idx, surname in enumerate(bien):
        if best_ratio == None:
            best_ratio = fuzz.ratio(name, surname)
            best_idx = 0
        else:
            ratio = fuzz.ratio(name, surname)
            if  ratio > best_ratio:
                best_ratio = ratio
                best_idx = idx
    map_list.append(bien[best_idx]) # obtain surname

Clientes['Provincia_Nombre'] = pd.Series(map_list) # add column
#dico = dico[["Name", "Surname", "Age", "Studies"]] # reorder columns

In [83]:
Clientes

,Id_Cliente,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,Longitud,Latitud,Provincia_Nombre
0,1,Buenos Aires,HEBER JONI SANTANA,LAS HERAS Y BAT. 24 DE FEBRERO 4150 RINCON DE...,42-5161,58.0,Loma verde,-58.818503,-34.309971,Buenos Aires
1,2,Buenos Aires,ANA SAPRIZA,PUEYRREDON Y DUPUY RUTA3 KM 52.500 S/N BÂº LO...,49-7578,61.0,Santa rosa,-58.730738,-34.939083,Buenos Aires
2,3,Buenos Aires,FERNANDO LUIS SARALEGUI,CALDERON DE LA BARCA 498,49-3435,15.0,Torres,-59.127941,-34.430822,Buenos Aires
3,4,Buenos Aires,MANUELA SARASOLA,"RUTA 36 KM 45, 500 S/N EL PELIGRO",49-2883,29.0,Ruta sol,-58.143940,-34.920527,Buenos Aires
4,5,Buenos Aires,MARIO RAÚL SARASUA,492 Y 186 S/N COLONIA URQUIZA,491-4608,34.0,Jose melchor romero,-58.893810,-34.944447,Buenos Aires
...,...,...,...,...,...,...,...,...,...,...
283,2626,Córdoba,ALICIA MARILEN PETROCELLI,CALLE PUBLICA S/NÂº LA CARBONADA RUTA 9 SUR KM...,156786618,19.0,Cordoba,NaN,NaN,Buenos Aires
284,2636,Córdoba,RODOLFO WASHINGTON PICCINI,CALLE PUBLICA SOL NACIENTE BARRIO SOL NACIENTE,NaN,44.0,Cordoba,NaN,NaN,Buenos Aires
285,2637,Córdoba,ALFREDO PICERNO,Calle publica LA FLORESTA SUR BARRIO LA FLORE...,4900752,21.0,Cordoba,NaN,NaN,Buenos Aires
286,2639,Córdoba,DIEGO ENRIQUE PIÑEIRO,MONSEÃ?OR D'ANDREA 1044 ALTO ALBERDI BÂ° ALTO ...,4347875,15.0,Cordoba,NaN,NaN,Buenos Aires


In [ ]:
# correccion provincias
nombres=[]
proporcion=[]
def cambios ( mal, bien):
    for i in mal:
        x=process.extractOne(i,bien)
        nombres.append(x[0])
        proporcion.append(x[1])
    return(nombres, proporcion)
mal = Clientes['Provincia'].dropna().values
bien = Provincia['Provincia_Nombre'].dropna().values
#Lista_provincias['Nombre_provincias']
#Provincia['Provincia_Nombre']
corregido , rango=cambios(mal, bien)

# se crea una nueva columna para el data frame
Clientes['Provincia_Nombre']=pd.Series(corregido)

Clientes